In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect("mydb.db")

In [3]:
connection

In [4]:
table_creation_query="""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    hire_date TEXT NOT NULL,
    salary REAL NOT NULL
);
"""

In [5]:
table_creation_query2="""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phone TEXT
);
"""

In [6]:
table_creation_query3="""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    amount REAL NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);

"""

In [7]:
cursor=connection.cursor()

In [8]:
cursor.execute(table_creation_query)
cursor.execute(table_creation_query2)
cursor.execute(table_creation_query3)

In [9]:
insert_query = """
INSERT INTO employees (emp_id, first_name, last_name, email, hire_date, salary)
VALUES (?, ?, ?, ?, ?, ?);
"""

insert_query_customers = """
INSERT INTO customers (customer_id, first_name, last_name, email, phone)
VALUES (?, ?, ?, ?, ?);
"""

insert_query_orders = """
INSERT INTO orders (order_id, customer_id, order_date, amount)
VALUES (?, ?, ?, ?);
"""

In [10]:
employee_data = [
    (1, "Amarnath", "Dhinakaran", "amar.jr@abc.com", "2023-06-01", 50000.00),
    (2, "Jasvee", "Adithri", "jasvee.a@gmail.com", "2022-04-15", 60000.00),
    (3, "Ranjani", "Amarnath", "ranji.a@jpg.com", "2021-09-30", 55000.00),
    (4, "Suba", "Seetharaman", "Suba.s@uio.com", "2020-01-20", 45000.00),
    ]

customers_data = [
    (1, "John", "Doe", "john.doe@example.com", "1234567890"),
    (2, "Jane", "Smith", "jane.smith@example.com", "9876543210"),
    (3, "Emily", "Davis", "emily.davis@example.com", "4567891230"),
    (4, "Michael", "Brown", "michael.brown@example.com", "7894561230"),
]

orders_data = [
    (1, 1, "2023-12-01", 250.75),
    (2, 2, "2023-11-20", 150.50),
    (3, 3, "2023-11-25", 300.00),
    (4, 4, "2023-12-02", 450.00),
]

In [11]:
cursor.executemany(insert_query,employee_data)
cursor.executemany(insert_query_customers,customers_data)
cursor.executemany(insert_query_orders,orders_data)

In [12]:
connection.commit()

In [13]:
cursor.execute("select * from orders;")

In [14]:
for row in cursor.fetchall():
    print(row)

(1, 1, '2023-12-01', 250.75)
(2, 2, '2023-11-20', 150.5)
(3, 3, '2023-11-25', 300.0)
(4, 4, '2023-12-02', 450.0)


In [15]:
#on top of this database iam going to create agents using agentic flow
from langchain_community.utilities import SQLDatabase

In [17]:
db = SQLDatabase.from_uri("sqlite:///mydb.db")

In [18]:
db

In [19]:
db.dialect

'sqlite'

In [21]:
db.get_usable_table_names()

['customers', 'employees', 'orders']

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

In [23]:
llm.invoke('ISRO')

AIMessage(content="ISRO (Indian Space Research Organisation) is the national space agency of India, responsible for the country's space program. Here are some key facts about ISRO:\n\n**History**: ISRO was established on August 15, 1969, with the vision of harnessing space technology for national development.\n\n**Objectives**: ISRO's primary objectives are:\n\n1. To develop and launch satellites for communication, navigation, and Earth observation.\n2. To develop and launch launch vehicles to carry satellites into space.\n3. To conduct space science and planetary exploration missions.\n4. To develop and demonstrate new space technologies.\n\n**Achievements**:\n\n1. **Mars Orbiter Mission (Mangalyaan)**: ISRO successfully launched India's first interplanetary mission to Mars in 2013, which is still operational.\n2. **Chandrayaan-1**: ISRO launched India's first lunar mission in 2008, which discovered water on the Moon.\n3. **Record-breaking satellite launches**: ISRO has launched a rec

In [24]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [25]:
toolkit = SQLDatabaseToolkit(db=db,llm=llm)

In [26]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [27]:
tools = toolkit.get_tools()

for i in tools:
    print(i)

description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields." db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>
description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3' db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>
db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>
description='Use this tool to double check if your query is correct before executing it. Always use this tool before execut

In [28]:
tools = toolkit.get_tools()

for i in tools:
    print(i.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [29]:
list_tables_tool = next((tool for tool in tools if tool.name == "sql_db_list_tables"),None)

In [30]:
list_tables_tool

ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>)

In [31]:
list_schema_tool = next((tool for tool in tools if tool.name == "sql_db_schema"),None)

In [32]:
list_schema_tool

InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001F45C5205E0>)

In [33]:
list_tables_tool.invoke("")

'customers, employees, orders'

In [35]:
print(list_schema_tool.invoke("customers"))


CREATE TABLE customers (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customers table:
customer_id	first_name	last_name	email	phone
1	John	Doe	john.doe@example.com	1234567890
2	Jane	Smith	jane.smith@example.com	9876543210
3	Emily	Davis	emily.davis@example.com	4567891230
*/


In [36]:
#create one more tool (custom tool)
from langchain_core.tools import tool
@tool
def db_query_tool(query:str)->str:

    """
    Execute a SQL query against the database and return the result.
    If the query is invalid or returns no result, an error message will be returned.
    In case of an error, the user is advised to rewrite the query and try again.
    """

    result = db.run_no_throw(query)
    if not result:
        return "Error : Query failed. Please rewrite your query and try again."
    return result

In [37]:
db_query_tool.invoke("SELECT * FROM Employees;")

"[(1, 'Amarnath', 'Dhinakaran', 'amar.jr@abc.com', '2023-06-01', 50000.0), (2, 'Jasvee', 'Adithri', 'jasvee.a@gmail.com', '2022-04-15', 60000.0), (3, 'Ranjani', 'Amarnath', 'ranji.a@jpg.com', '2021-09-30', 55000.0), (4, 'Suba', 'Seetharaman', 'Suba.s@uio.com', '2020-01-20', 45000.0)]"

In [38]:
db.run("SELECT * FROM Employees;")

"[(1, 'Amarnath', 'Dhinakaran', 'amar.jr@abc.com', '2023-06-01', 50000.0), (2, 'Jasvee', 'Adithri', 'jasvee.a@gmail.com', '2022-04-15', 60000.0), (3, 'Ranjani', 'Amarnath', 'ranji.a@jpg.com', '2021-09-30', 55000.0), (4, 'Suba', 'Seetharaman', 'Suba.s@uio.com', '2020-01-20', 45000.0)]"